In [0]:
#Libraries management
from pyspark import pipelines as pl
from pyspark.sql.functions import *
from pyspark.sql.types import *
 
volume_path = "/Volumes/workspace/damg7370/datastore/schemadrift_addnewcolumns/customer_*.json"

In [0]:
pl.create_streaming_table("demo_cust_bronze_addcols")
 
@pl.append_flow(

    target="demo_cust_bronze_addcols",

    name="demo_cust_bronze_addcols_ingest_flow"

)

def demo_cust_bronze_addcols_ingest_flow():

    df = (

        spark.readStream

            .format("cloudFiles")

            .option("cloudFiles.format", "json")

            .option("cloudFiles.inferColumnTypes", "true")

            .option("cloudFiles.schemaEvolutionMode", "addNewColumns")   # <-- IMPORTANT

            .load(volume_path)

    )

    return (

        df.withColumn("ingestion_datetime", current_timestamp())

          .withColumn("source_filename", col("_metadata.file_path"))

    )

 

In [0]:
# For structural consistency only — NOT USED in addNewColumns
def process__rescue_data_new_fields(df):
    return df
 
def process__rescue_data_datatype_change(df, target_schema):
    return df

In [0]:
updated_datatypes = StructType([
    StructField("signupDate", DateType(), True)
])

In [0]:
pl.create_streaming_table(
    name="demo_cust_silver_addcols",
    expect_all_or_drop={"valid_id": "CustomerID IS NOT NULL"}
)
@pl.append_flow(
    target="demo_cust_silver_addcols",
    name="demo_cust_silver_addcols_clean_flow"
)
def demo_cust_silver_addcols_clean_flow():
    df = spark.readStream.table("demo_cust_bronze_addcols")
 
    # Calls below do nothing but keep structure consistent
    df = process__rescue_data_new_fields(df)
    df = process__rescue_data_datatype_change(df, updated_datatypes)
 
    return df